# PR Sentiment Analysis

## Import Library

In [1]:
import numpy as np
import pandas as pd

# library for stopwords, stemmer
!pip install Sastrawi

# library for entity masking
!pip install stanza

# library for entity masking
!pip install transformers

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_vendor\packaging\version.py", line 56, in parse
    return Version(version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_v

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_vendor\packaging\version.py", line 56, in parse
    return Version(version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_v

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_vendor\packaging\version.py", line 56, in parse
    return Version(version)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\pip\_v

## Import Dataset

In [2]:
train_data = pd.read_csv("data/train_preprocess.tsv", sep='\t', names=['sentence', 'sentiment'])
test_data = pd.read_csv("data/test_preprocess.tsv", sep='\t', names=['sentence', 'sentiment'])

train_data.head()

,sentence,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [3]:
print(f"Ukuran data train:", train_data.shape)
print(f"Ukuran data test:,", test_data.shape)

Ukuran data train: (11000, 2)
Ukuran data test:, (500, 2)


## Preprocessing (train and test data)

Preprocessing yang dilakukan adalah :
1. Removing character other than alphabet
2. Lowercase
3. Dropping Stopwords
4. Stemming
5. Entity Masking (belum sempat diimplement karena library yang dipakai bermasalah)

In [30]:
import re
import nltk
# import stanza
# stanza.download('id')
from transformers import pipeline
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def preprocess_data(data):
    
    # function for entity masking
    
    def entity_masking(text):
        masked_text = text
#         doc = nlp(text)
        ner_results = ner(text)
        for entity in ner_results:
            entity_type = entity.type
            entity_text = entity.text

            # Replace the entity with a mask based on its type
            if entity_type == 'PERSON':
                masked_text = masked_text.replace(entity_text, '[PERSON]')
            elif entity_type == 'ORG':
                masked_text = masked_text.replace(entity_text, '[ORG]')
            elif entity_type == 'LOC':
                masked_text = masked_text.replace(entity_text, '[LOC]')
        return masked_text
    
    # create a stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # create a stopwords remover
    stop_factory = StopWordRemoverFactory()

    # additional stopwords
    more_stopword = []

    # stopwords from the factory and additional stopwords combined
    stopwords = stop_factory.get_stop_words() + more_stopword
    
    corpus = []

    for i in range(0, len(data)):

        # removing any character other than alphabet
        sentence = re.sub('[^a-zA-Z]', ' ', data['sentence'][i])

        # lowercase the sentence
        sentence = sentence.lower()
        
        # remove stopwords
        sentence = sentence.split()
        sentence = [word for word in sentence if word not in stopwords]
        sentence = ' '.join(sentence)

        # entity masking
#         masked_text = entity_masking(sentence)

        # stem the word 
        stemmed_text = stemmer.stem(sentence)

        # add the stemmed sentence to corpus
        corpus.append(stemmed_text)
    
    return corpus

In [31]:
preprocessed_train_data = preprocess_data(train_data)
preprocessed_test_data = preprocess_data(test_data)

## Raw vs Preprocessed

In [32]:
# original sentence
print("ORIGINAL:")
print(train_data['sentence'][0])

print()

# preprocessed sentence
print("PREPROCESSED:")
print(preprocessed_train_data[0])

ORIGINAL:
warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !

PREPROCESSED:
warung milik usaha pabrik tahu puluh tahun kenal buat tahu putih bandung tahu kualitas padu ahli masak padu kretivitas jadi warung saji menu utama bahan tahu tambah menu umum ayam semua selera indonesia harga cukup jangkau jangan lewat tahu bletoka nya kalah asli tegal


## Training and Test Data

### Variation of Feature with Bag of Words

In [33]:
# bag of words with boolean as word feature value
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500, binary=True)

In [34]:
# bag of words with TF as word feature value
cv_tf = CountVectorizer(max_features=500, binary=False)

In [35]:
# bag of words with TF-IDF as word feature value
from sklearn.feature_extraction.text import TfidfVectorizer
cv_tfidf = TfidfVectorizer(max_features=500)

### Feature (X) and Label Data (y) with each Variation

In [36]:
# label is the same for all variation
y_train = train_data.iloc[:, -1].values
y_test = test_data.iloc[:, -1].values

In [37]:
# X with boolean bag of words
X_bool_train = cv.fit_transform(preprocessed_train_data).toarray()
X_bool_test = cv.fit_transform(preprocessed_test_data).toarray()

In [38]:
# X with TF bag of words
X_tf_train = cv_tf.fit_transform(preprocessed_train_data).toarray()
X_tf_test = cv_tf.fit_transform(preprocessed_test_data).toarray()

In [39]:
# X with TF-IDF bag of words
X_tf_idf_train = cv_tfidf.fit_transform(preprocessed_train_data).toarray()
X_tf_idf_test = cv_tfidf.fit_transform(preprocessed_test_data).toarray()

## Experiment

Variasi algoritma yang dipilih untuk eksperimen adalah : 
1. Naive Bayes
2. Logistic Regression
3. Support Vector Machine
4. k-Nearest Neighbors dengan n_neighbors = 5

In [40]:
print(f"Ukuran train data:", train_data.shape)
print(f"Ukuran test data:", test_data.shape)

Ukuran train data: (11000, 2)
Ukuran test data: (500, 2)


In [41]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, classification_report, confusion_matrix

def metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    cm = confusion_matrix(y_test, y_pred)

    print("Model:", classifier, "\n")
    print(f"Accuracy Score  : {accuracy:.2}")
    print(f"Precision Score : {precision:.2}")
    print(f"Recall Score    : {recall:.2}")
    print(f"F1 Score        : {f1:.2}")
    print()
    print(classification_report(y_test, y_pred))
    print()
    print("Confusion Matrix:")
    print(cm)

### Bag of Words Variation with Boolean

In [42]:
X_train = X_bool_train
X_test = X_bool_test

#### Naive Bayes

In [76]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: MultinomialNB() 

Accuracy Score  : 0.44
Precision Score : 0.36
Recall Score    : 0.36
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.46      0.31      0.37       204
     neutral       0.18      0.05      0.07        88
    positive       0.44      0.72      0.55       208

    accuracy                           0.44       500
   macro avg       0.36      0.36      0.33       500
weighted avg       0.40      0.44      0.39       500


Confusion Matrix:
[[ 64  11 129]
 [ 24   4  60]
 [ 51   7 150]]


#### Logistic Regression

In [77]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LogisticRegression() 

Accuracy Score  : 0.38
Precision Score : 0.35
Recall Score    : 0.33
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.36      0.41      0.38       204
     neutral       0.29      0.12      0.17        88
    positive       0.40      0.45      0.43       208

    accuracy                           0.38       500
   macro avg       0.35      0.33      0.33       500
weighted avg       0.37      0.38      0.36       500


Confusion Matrix:
[[ 83  16 105]
 [ 42  11  35]
 [103  11  94]]


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Support Vector Machine (SVM)

In [78]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LinearSVC() 

Accuracy Score  : 0.38
Precision Score : 0.36
Recall Score    : 0.33
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.37      0.38      0.37       204
     neutral       0.32      0.14      0.19        88
    positive       0.40      0.49      0.44       208

    accuracy                           0.38       500
   macro avg       0.36      0.33      0.33       500
weighted avg       0.37      0.38      0.37       500


Confusion Matrix:
[[ 77  14 113]
 [ 36  12  40]
 [ 96  11 101]]


#### k-Nearest Neighbors (k-NN)

In [65]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: KNeighborsClassifier() 

Accuracy Score  : 0.35
Precision Score : 0.32
Recall Score    : 0.32
F1 Score        : 0.31

              precision    recall  f1-score   support

    negative       0.40      0.54      0.46       204
     neutral       0.19      0.22      0.20        88
    positive       0.37      0.21      0.26       208

    accuracy                           0.35       500
   macro avg       0.32      0.32      0.31       500
weighted avg       0.35      0.35      0.33       500


Confusion Matrix:
[[111  43  50]
 [ 45  19  24]
 [125  40  43]]


### Bag of Words Variation with Term Frequency

In [66]:
X_train = X_tf_train
X_test = X_tf_test

#### Naive Bayes

In [67]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: MultinomialNB() 

Accuracy Score  : 0.39
Precision Score : 0.37
Recall Score    : 0.37
F1 Score        : 0.37

              precision    recall  f1-score   support

    negative       0.41      0.48      0.44       204
     neutral       0.26      0.27      0.26        88
    positive       0.44      0.36      0.40       208

    accuracy                           0.39       500
   macro avg       0.37      0.37      0.37       500
weighted avg       0.40      0.39      0.39       500


Confusion Matrix:
[[ 98  42  64]
 [ 34  24  30]
 [105  28  75]]


#### Logistic Regression

In [68]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LogisticRegression() 

Accuracy Score  : 0.39
Precision Score : 0.36
Recall Score    : 0.35
F1 Score        : 0.35

              precision    recall  f1-score   support

    negative       0.41      0.57      0.48       204
     neutral       0.29      0.18      0.22        88
    positive       0.39      0.31      0.34       208

    accuracy                           0.39       500
   macro avg       0.36      0.35      0.35       500
weighted avg       0.38      0.39      0.38       500


Confusion Matrix:
[[116  17  71]
 [ 43  16  29]
 [122  22  64]]


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Support Vector Machine (SVM)

In [69]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LinearSVC() 

Accuracy Score  : 0.4
Precision Score : 0.39
Recall Score    : 0.36
F1 Score        : 0.36

              precision    recall  f1-score   support

    negative       0.41      0.58      0.48       204
     neutral       0.36      0.18      0.24        88
    positive       0.40      0.32      0.35       208

    accuracy                           0.40       500
   macro avg       0.39      0.36      0.36       500
weighted avg       0.40      0.40      0.39       500


Confusion Matrix:
[[119  14  71]
 [ 44  16  28]
 [127  15  66]]


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


#### k-Nearest Neighbors (k-NN)

In [70]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: KNeighborsClassifier() 

Accuracy Score  : 0.33
Precision Score : 0.33
Recall Score    : 0.34
F1 Score        : 0.27

              precision    recall  f1-score   support

    negative       0.40      0.60      0.48       204
     neutral       0.19      0.35      0.24        88
    positive       0.39      0.05      0.09       208

    accuracy                           0.33       500
   macro avg       0.33      0.34      0.27       500
weighted avg       0.36      0.33      0.28       500


Confusion Matrix:
[[123  70  11]
 [ 51  31   6]
 [132  65  11]]


### Bag of Words Variation with TF-IDF

In [79]:
X_train = X_tf_idf_train
X_test = X_tf_idf_test

#### Naive Bayes

In [80]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: MultinomialNB() 

Accuracy Score  : 0.44
Precision Score : 0.36
Recall Score    : 0.36
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.46      0.31      0.37       204
     neutral       0.18      0.05      0.07        88
    positive       0.44      0.72      0.55       208

    accuracy                           0.44       500
   macro avg       0.36      0.36      0.33       500
weighted avg       0.40      0.44      0.39       500


Confusion Matrix:
[[ 64  11 129]
 [ 24   4  60]
 [ 51   7 150]]


#### Logistic Regression

In [81]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LogisticRegression() 

Accuracy Score  : 0.38
Precision Score : 0.35
Recall Score    : 0.33
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.36      0.41      0.38       204
     neutral       0.29      0.12      0.17        88
    positive       0.40      0.45      0.43       208

    accuracy                           0.38       500
   macro avg       0.35      0.33      0.33       500
weighted avg       0.37      0.38      0.36       500


Confusion Matrix:
[[ 83  16 105]
 [ 42  11  35]
 [103  11  94]]


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Support Vector Machine (SVM)

In [82]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: LinearSVC() 

Accuracy Score  : 0.38
Precision Score : 0.36
Recall Score    : 0.33
F1 Score        : 0.33

              precision    recall  f1-score   support

    negative       0.37      0.38      0.37       204
     neutral       0.32      0.14      0.19        88
    positive       0.40      0.49      0.44       208

    accuracy                           0.38       500
   macro avg       0.36      0.33      0.33       500
weighted avg       0.37      0.38      0.37       500


Confusion Matrix:
[[ 77  14 113]
 [ 36  12  40]
 [ 96  11 101]]


#### k-Nearest Neighbors (k-NN)

In [83]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

metrics(y_test, y_pred)

Model: KNeighborsClassifier() 

Accuracy Score  : 0.35
Precision Score : 0.32
Recall Score    : 0.32
F1 Score        : 0.31

              precision    recall  f1-score   support

    negative       0.40      0.54      0.46       204
     neutral       0.19      0.22      0.20        88
    positive       0.37      0.21      0.26       208

    accuracy                           0.35       500
   macro avg       0.32      0.32      0.31       500
weighted avg       0.35      0.35      0.33       500


Confusion Matrix:
[[111  43  50]
 [ 45  19  24]
 [125  40  43]]
